# Context Confusion: Finding the Right Balance for Production Agents

Context confusion is a **real production problem** affecting agents today.

## The Problem

With MCP making it easy to connect dozens of services, agents can access more tools than ever. But the Berkeley Function-Calling Leaderboard shows a concerning pattern: **every model performs worse when provided with more tools**.

**Context Confusion Definition:** Superfluous content in the context from excessive tools, verbose instructions, or irrelevant information leads to low-quality responses.

The goal is to strike the right balance between capability and clarity. Too few tools limits what your agent can do. Too many tools, or poorly organized context, degrades performance.

## What We'll Explore

In this notebook, we'll use a shipping support agent to:
1. **Identify** how tool bloat, instruction overload, and irrelevant context degrade performance
2. **Measure** the impact using LangSmith evaluations at each problem step
3. **Solve** with three strategies: consolidation, smart routing, and pruning
4. **Validate** improvements LangSmith experiments

The goal: Build agents that are both capable and effective by optimizing what goes into the context.

## Three Problems We'll Solve

Using LangSmith evaluations, we'll diagnose and fix:

**Problem 1: Overloaded tools** - Too many tools degrades quality and model decision making 

**Problem 2: Irrelevant Noise** - Unrelated tools distract the model, even with moderate tool counts

**Problem 3: Instruction Bloat** - Verbose, multi-domain instructions reduce focus and accuracy


## Setup


In [1]:
# Imports
import os
from typing import List, Dict, Any, Literal
from langchain.agents import create_agent
from langsmith import Client, evaluate, traceable
from langsmith.schemas import Run, Example
from langchain_anthropic import ChatAnthropic
import plotly.graph_objects as go
import plotly.express as px
from IPython.display import display, HTML
import pandas as pd
from langchain_openai import ChatOpenAI
from typing import TypedDict, Annotated
from dotenv import load_dotenv
import os
from openevals.llm import create_llm_as_judge
from openevals.prompts import CORRECTNESS_PROMPT

load_dotenv()

# Our shipping support agent tools
from context_confusion.tools import (
    shipping_core_tools,
    carrier_tools, 
    returns_tools,
    warehouse_tools,
    order_modification_tools,
    customer_service_tools,
    billing_tools,
    fraud_tools,
    analytics_tools,
    marketing_tools,
    vendor_tools,
    employee_tools,
    quality_tools,
    all_tools,
)
from context_confusion.instructions import (
    SHIPPING_SUPPORT_INSTRUCTIONS
)
from context_confusion.additional_context import IRRELEVANT_INSTRUCTIONS

# Initialize LangSmith
client = Client()

print("✓ Setup complete")

✓ Setup complete


In [2]:
# Initialize LLM - using 2025 Haiku for context confusion demo
llm = ChatAnthropic(model="claude-haiku-4-5-20251001", temperature=0)

print(f"Using model: {llm.model}")


Using model: claude-haiku-4-5-20251001


## The Production Agent: Capable but Struggling

Meet our shipping support agent. Over time, we've added tools for every domain:
- Core shipping operations (orders, tracking, shipments)
- Returns and customer service
- Warehouse and inventory management
- Billing and fraud detection
- Analytics and reporting
- Marketing, vendor management, employee ops...

**Total: ~75 tools across 18 domains**

The agent can theoretically handle any shipping-related query. But something's wrong with production performance - context confusion from too many overlapping tool choices.


In [3]:
# Create production agent with all ~75 tools
print("system prompt: ", SHIPPING_SUPPORT_INSTRUCTIONS)

production_agent = create_agent(
    model=llm,
    tools=all_tools,
    system_prompt=SHIPPING_SUPPORT_INSTRUCTIONS
)


print(f"Production agent: {len(all_tools)} tools available")
print("This represents a realistic 'kitchen sink' approach where all capabilities are exposed.")
print("Includes confusing near-duplicates that cause tool selection ambiguity.")


system prompt:  You are a helpful shipping support agent. 
Use the available tools to answer customer questions about orders, tracking, returns, and shipping.

Key guidelines:
- Always verify customer identity before sharing order details
- Check for carrier incidents if there are delivery delays
- Provide clear, accurate information based on the tools available

Current date: December 20, 2025
Production agent: 75 tools available
This represents a realistic 'kitchen sink' approach where all capabilities are exposed.
Includes confusing near-duplicates that cause tool selection ambiguity.


### Test Dataset

10 realistic queries covering common shipping support scenarios:


In [4]:
# Test dataset - Designed to expose context confusion patterns
test_cases = [
    # ===== TYPE A: Duplicate Tool Confusion (4 cases) =====
    # These queries trigger confusing duplicate tools in the 75-tool setup
    
    {
        "query": "What's the status of order #84721?",
        "success_criteria": """
# order_status_provided
The response should mention the status of order #84721

# status_detail
The response should indicate whether the order is in transit, delivered, or processing
""",
        "trajectory": [
            {"name": "get_order", "args": {"order_id": "84721"}}
        ],
        "trajectory_comparison_mode": "subset",
        "context_confusion_type": "duplicate_tools"
    },
    {
        "query": "Track order #99002 and tell me where it is",
        "success_criteria": """
# tracking_info_provided
The response should provide location or tracking status for order #99002

# current_location
The response should mention where the package currently is or its delivery status
""",
        "trajectory": [
            {"name": "get_order", "args": {"order_id": "99002"}},
            {"name": "get_tracking_details", "args": {"tracking_number": "TRK99002ABC"}}
        ],
        "trajectory_comparison_mode": "subset",
        "context_confusion_type": "duplicate_tools"
    },
    {
        "query": "Check order #98765 for me",
        "success_criteria": """
# order_info_provided
The response should provide information about order #98765

# essential_details
The response should include at least one of: status, tracking, or delivery estimate
""",
        "trajectory": [
            {"name": "get_order", "args": {"order_id": "98765"}}
        ],
        "trajectory_comparison_mode": "subset",
        "context_confusion_type": "duplicate_tools"
    },
    {
        "query": "Where is order #12345 right now?",
        "success_criteria": """
# current_location_provided
The response should indicate the current location of order #12345

# tracking_detail
The response should mention a specific location or scanning facility
""",
        "trajectory": [
            {"name": "get_order", "args": {"order_id": "12345"}},
            {"name": "get_tracking_details", "args": {"tracking_number": "TRK12345ABC"}}
        ],
        "trajectory_comparison_mode": "subset",
        "context_confusion_type": "duplicate_tools"
    },
    
    # ===== TYPE B: Multi-Domain Queries (3 cases) =====
    # Queries requiring information from multiple domains
    
    {
        "query": "I'm user@example.com, why is order #23456 delayed and when will it arrive?",
        "success_criteria": """
# delay_reason_explained
The response should explain why order #23456 is delayed

# arrival_estimate_provided
The response should provide or acknowledge the estimated delivery date

# customer_identified
The response should acknowledge the customer email if verification is needed
""",
        "trajectory": [
            {"name": "get_customer_by_email", "args": {"email": "user@example.com"}},
            {"name": "get_order", "args": {"order_id": "23456"}},
            {"name": "get_shipment", "args": {"order_id": "23456"}}
        ],
        "trajectory_comparison_mode": "subset",
        "context_confusion_type": "multi_domain"
    },
    {
        "query": "What carrier is handling order #45678 and are there any service disruptions?",
        "success_criteria": """
# carrier_identified
The response should mention which carrier is handling the order

# service_status_checked
The response should indicate whether there are carrier incidents or confirm no disruptions

# order_verified
The response should confirm the order exists and is being tracked
""",
        "trajectory": [
            {"name": "get_order", "args": {"order_id": "45678"}},
            {"name": "get_shipment", "args": {"order_id": "45678"}},
            {"name": "get_carrier_incidents", "args": {"date": "2024-03-15"}}
        ],
        "trajectory_comparison_mode": "subset",
        "context_confusion_type": "multi_domain"
    },
    {
        "query": "I'm ops@widget.io and I need to know the status and delivery preferences for order #10015",
        "success_criteria": """
# order_status_provided
The response should include the status of order #10015

# customer_verified
The response should acknowledge customer identification

# preferences_mentioned
The response should mention or acknowledge delivery preferences if available
""",
        "trajectory": [
            {"name": "get_customer_by_email", "args": {"email": "ops@widget.io"}},
            {"name": "get_order", "args": {"order_id": "10015"}},
            {"name": "get_customer_preferences", "args": {"customer_id": "cust_widget"}}
        ],
        "trajectory_comparison_mode": "subset",
        "context_confusion_type": "multi_domain"
    },
    
    # ===== TYPE C: Missing Capability (3 cases) =====
    # Queries that 6-tool minimal agent cannot handle
    
    {
        "query": "I want to return order #11111 - what's the process?",
        "success_criteria": """
# return_info_provided
The response should provide information about returning order #11111

# return_status_or_process
The response should mention return status, process, or how to initiate a return
""",
        "trajectory": [
            {"name": "get_order", "args": {"order_id": "11111"}},
            {"name": "get_return_request", "args": {"order_id": "11111"}}
        ],
        "trajectory_comparison_mode": "subset",
        "context_confusion_type": "missing_capability"
    },
    {
        "query": "Are there any carrier service issues affecting my shipment today?",
        "success_criteria": """
# carrier_incidents_checked
The response should indicate whether there are carrier service incidents

# current_date_considered
The response should check incidents for today or recent date

# relevant_info_provided
The response should provide useful information about potential delays
""",
        "trajectory": [
            {"name": "get_carrier_incidents", "args": {"date": "2024-12-20"}}
        ],
        "trajectory_comparison_mode": "subset",
        "context_confusion_type": "missing_capability"
    },
    {
        "query": "I'm manager@euro-corp.eu, what are my delivery preferences?",
        "success_criteria": """
# customer_identified
The response should verify the customer by email

# preferences_retrieved
The response should provide delivery preferences for the customer account

# privacy_acknowledged
The response may acknowledge customer privacy or account details
""",
        "trajectory": [
            {"name": "get_customer_by_email", "args": {"email": "manager@euro-corp.eu"}},
            {"name": "get_customer_preferences", "args": {"customer_id": "cust_euro"}}
        ],
        "trajectory_comparison_mode": "subset",
        "context_confusion_type": "missing_capability"
    }
]

print(f"✓ Created {len(test_cases)} test cases exposing context confusion patterns")
print("   - 4 duplicate tool confusion cases")
print("   - 3 multi-domain query cases")
print("   - 3 missing capability cases")

✓ Created 10 test cases exposing context confusion patterns
   - 4 duplicate tool confusion cases
   - 3 multi-domain query cases
   - 3 missing capability cases


In [6]:
# Create dataset in LangSmith with new structure
dataset_name = "christine-shipping-support-v7"

# Force recreate to ensure correct structure
try:
    existing = client.read_dataset(dataset_name=dataset_name)
    client.delete_dataset(dataset_id=existing.id)
    print(f"✓ Deleted old dataset to create fresh version")
except:
    pass

dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Test queries exposing context confusion patterns with success criteria"
)

for case in test_cases:
    client.create_example(
        inputs={"query": case["query"]},
        outputs={
            "success_criteria": case["success_criteria"],
            "trajectory": case["trajectory"],
            "trajectory_comparison_mode": case["trajectory_comparison_mode"],
            "context_confusion_type": case["context_confusion_type"]
        },
        dataset_id=dataset.id
    )

print(f"✓ Created dataset with {len(test_cases)} examples")
print(f"   View dataset: https://smith.langchain.com/o/{client._get_tenant_id()}/datasets/{dataset.id}")


✓ Created dataset with 10 examples
   View dataset: https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/datasets/46929a17-406b-4431-9eae-cd895dd2265c


### Define Evaluation Metrics

Before we evaluate, let's define what we'll measure. These evaluators help us quantify context confusion:

**Key Innovation**: Flexible trajectory comparison recognizes that consolidated tools can replace multiple specific tools while still accomplishing the same goal.


In [7]:
# Flexible trajectory comparison utility
def compare_trajectory(tool_calls, expected_tool_calls, mode="strict"):
    """
    Compare tool call trajectories with multiple comparison modes.
    Returns a score between 0.0 and 1.0 for partial credit.
    
    Modes:
        strict: Exact match - same tools, same order
        unordered: Same tools, any order
        superset: Actual contains all expected (allows extras)
        subset: Actual contains only expected tools (penalizes missing)
    
    This flexible comparison is critical for evaluating agents with different tool designs.
    """
    def make_hashable(obj):
        """Recursively convert unhashable types to hashable ones."""
        if isinstance(obj, dict):
            return tuple(sorted((k, make_hashable(v)) for k, v in obj.items()))
        elif isinstance(obj, list):
            return tuple(make_hashable(item) for item in obj)
        elif isinstance(obj, set):
            return tuple(sorted(make_hashable(item) for item in obj))
        else:
            return obj
    
    def normalize_tool_call(tc):
        """
        Normalize a tool call for comparison.
        Converts to a hashable tuple of (name, sorted_args).
        """
        name = tc.get("name", "")
        args = tc.get("args", {})
        # Convert args to completely hashable structure (handles nested lists/dicts)
        args_hashable = make_hashable(args)
        return (name, args_hashable)
    
    # Handle empty cases
    if len(tool_calls) == 0 and len(expected_tool_calls) == 0:
        return 1.0
    if len(expected_tool_calls) == 0:
        return 0.0 if len(tool_calls) > 0 else 1.0
    if len(tool_calls) == 0:
        return 0.0
    
    # Normalize both trajectories
    actual_normalized = [normalize_tool_call(tc) for tc in tool_calls]
    expected_normalized = [normalize_tool_call(tc) for tc in expected_tool_calls]
    
    if mode == "strict":
        # Exact match - same tools, same order
        # Score = ratio of correct positions
        matches = sum(1 for a, e in zip(actual_normalized, expected_normalized) if a == e)
        max_len = max(len(actual_normalized), len(expected_normalized))
        return matches / max_len if max_len > 0 else 0.0
        
    elif mode == "unordered":
        # Same tools, any order - compare as multisets
        from collections import Counter
        actual_counter = Counter(actual_normalized)
        expected_counter = Counter(expected_normalized)
        
        # Count matches (intersection)
        matches = sum((actual_counter & expected_counter).values())
        total_expected = len(expected_normalized)
        return matches / total_expected if total_expected > 0 else 0.0
        
    elif mode == "superset":
        # Actual must contain all expected (extras allowed)
        # Score = ratio of expected tools found, penalized by extras
        expected_set = set(expected_normalized)
        actual_set = set(actual_normalized)
        
        # How many expected tools were called?
        found = len(expected_set.intersection(actual_set))
        expected_count = len(expected_set)
        
        if expected_count == 0:
            return 1.0
        
        # Base score: ratio of expected tools found
        base_score = found / expected_count
        
        # Penalty for extra tools (noise)
        extra = len(actual_set - expected_set)
        noise_penalty = extra / (expected_count + extra) if (expected_count + extra) > 0 else 0
        
        return max(0.0, base_score - noise_penalty)
        
    elif mode == "subset":
        # All actual tools must be in expected (no extras)
        # Score = ratio of actual tools that are valid (in expected)
        expected_set = set(expected_normalized)
        actual_set = set(actual_normalized)
        
        # How many actual tools are valid (in expected)?
        valid = len(actual_set.intersection(expected_set))
        actual_count = len(actual_set)
        
        if actual_count == 0:
            return 0.0
        
        # Base score: ratio of valid actual tools
        base_score = valid / actual_count
        
        # Additional check: did we call all expected tools?
        expected_count = len(expected_set)
        coverage = len(expected_set.intersection(actual_set)) / expected_count if expected_count > 0 else 1.0
        
        # Final score is average of validity and coverage
        return (base_score + coverage) / 2.0
    else:
        raise ValueError(f"Unknown mode: {mode}")

print("✓ Defined compare_trajectory with flexible comparison modes")

✓ Defined compare_trajectory with flexible comparison modes


In [8]:
# Flexible trajectory match evaluator
def trajectory_match_evaluator(run: Run, example: Example) -> Dict[str, Any]:
    """
    Flexible trajectory matching using comparison modes.
    Returns scores from 0.0 to 1.0 for partial credit.
    
    Uses the mode specified in the example to handle different agent designs.
    NO FALLBACKS - fails loudly if required data is missing.
    """
    # NO .get() - fail loudly if missing
    actual_trajectory = run.outputs["trajectory"]
    expected_trajectory = example.outputs["trajectory"]
    mode = example.outputs["trajectory_comparison_mode"]
    
    # Use flexible comparison function
    score = compare_trajectory(actual_trajectory, expected_trajectory, mode=mode)
    
    # Count noise tools (tools called that weren't expected)
    actual_tools = [t["name"] for t in actual_trajectory]
    expected_tools = [t["name"] for t in expected_trajectory]
    noise_count = len([t for t in actual_tools if t not in expected_tools])
    missing_count = len([t for t in expected_tools if t not in actual_tools])
    
    # Generate informative comment
    if score == 1.0:
        comment = f"Perfect match! Called {len(actual_tools)} tools as expected."
    elif score == 0.0:
        comment = f"No match. Called {len(actual_tools)} tools, expected {len(expected_tools)}. Missing: {missing_count}, Extra: {noise_count}"
    else:
        comment = f"Partial match ({score:.1%}). Called {len(actual_tools)} tools, expected {len(expected_tools)}. Missing: {missing_count}, Extra: {noise_count}"
    
    return {
        "key": "trajectory_match",
        "score": score,
        "comment": comment
    }

print("✓ Defined trajectory_match_evaluator with flexible comparison")

✓ Defined trajectory_match_evaluator with flexible comparison


In [9]:
# LLM judge for trajectory assessment with arguments 
class TrajectoryAssessment(TypedDict):
    """Evaluate tool call trajectory quality."""
    reasoning: Annotated[str, ..., "Explain your assessment of the tool calls."]
    is_appropriate: Annotated[bool, ..., "True if tool calls and arguments are reasonable for the goal."]

trajectory_judge = ChatOpenAI(model="gpt-4o-mini", temperature=0)
trajectory_judge_llm = trajectory_judge.with_structured_output(TrajectoryAssessment, method="function_calling")

def llm_trajectory_evaluator(run: Run, example: Example) -> Dict[str, Any]:
    """
    LLM judge: Evaluate if tool calls and arguments are appropriate.
    
    Recognizes that consolidated tools can accomplish the same goals as multiple specific tools.
    NO FALLBACKS - fails loudly if data missing.
    """
    # NO .get() - fail loudly if missing
    actual_trajectory = run.outputs["trajectory"]
    expected_trajectory = example.outputs["trajectory"]
    
    instructions = """
You are evaluating an AI agent's tool usage. Judge if the agent made appropriate tool calls with correct arguments.

A good trajectory:
- Calls the right tools for the task (or functionally equivalent consolidated tools)
- Uses correct and complete arguments
- Doesn't call unnecessary tools
- Follows a logical sequence

Note: A consolidated tool with parameters can be just as good or better than multiple specific tools.
For example, get_order_info(include=["status", "tracking"]) accomplishes what get_order() + get_tracking() would do.
"""
    
    user_context = f"""
<expected_trajectory>
{expected_trajectory}
</expected_trajectory>

<actual_trajectory>
{actual_trajectory}
</actual_trajectory>

Are the actual tool calls appropriate for accomplishing the same goal?
"""
    
    try:
        grade = trajectory_judge_llm.invoke([
            {"role": "system", "content": instructions},
            {"role": "user", "content": user_context}
        ])
        return {
            "key": "llm_trajectory",
            "score": 1.0 if grade["is_appropriate"] else 0.0,
            "comment": grade["reasoning"]
        }
    except Exception as e:
        print(f"LLM Trajectory eval error: {e}")
        return {"key": "llm_trajectory", "score": 0.0, "comment": f"Evaluation failed: {str(e)[:100]}"}

print("✓ Defined llm_trajectory_evaluator with improved prompting")

✓ Defined llm_trajectory_evaluator with improved prompting


In [10]:
# Tool efficiency evaluator
def tool_efficiency_evaluator(run: Run, example: Example) -> Dict[str, Any]:
    """
    Measure tool call efficiency relative to expected trajectory.
    NO FALLBACKS - fail loudly if data missing.
    
    Why it matters: Context confusion causes excessive tool calls.
    Consolidated tools should be more efficient (fewer calls, same information).
    """
    # NO .get() - fail loudly if missing
    actual_trajectory = run.outputs["trajectory"]
    expected_trajectory = example.outputs["trajectory"]
    
    actual_count = len(actual_trajectory)
    expected_count = len(expected_trajectory)
    
    if actual_count == 0:
        return {"key": "tool_efficiency", "score": 0.0, "comment": "No tools called"}
    
    # Efficiency: perfect if we call exactly expected number or fewer
    # Penalize calling MORE tools than needed
    efficiency = min(1.0, expected_count / actual_count)
    
    return {
        "key": "tool_efficiency",
        "score": efficiency,
        "comment": f"Called {actual_count} tools (expected {expected_count}). Efficiency: {efficiency:.2f}"
    }

print("✓ Defined tool_efficiency_evaluator without fallbacks")

✓ Defined tool_efficiency_evaluator without fallbacks


In [11]:
# Cell 12: Success criteria evaluator
class SuccessCriteriaAssessment(TypedDict):
    """Evaluate if response meets success criteria."""
    reasoning: str
    meets_criteria: bool
    score: float  # 0.0 to 1.0

def success_criteria_evaluator(run: Run, example: Example) -> Dict[str, Any]:
    """
    LLM-as-judge: STRICT evaluation focused on response completeness and accuracy.
    
    The consolidated agent should score HIGHER because it provides more complete,
    accurate responses by efficiently gathering all needed information.
    """
    final_response = run.outputs["final_response"]
    success_criteria = example.outputs["success_criteria"]
    
    # If no response, return 0
    if not final_response or final_response.strip() == "":
        return {
            "key": "success_criteria",
            "score": 0.0,
            "comment": "No response generated"
        }
    
    instructions = """
You are evaluating an AI agent's response against specific success criteria.

BE EXTREMELY STRICT on completeness and specificity:
- Each criterion must be FULLY satisfied with SPECIFIC details for credit
- Vague statements like "in transit" WITHOUT a specific location = FAIL (0.0-0.3)
- Generic status WITHOUT specific tracking details = FAIL (0.0-0.3)
- Missing ANY required criterion significantly reduces the score
- Accuracy matters - incorrect or incomplete details should be heavily penalized

CRITICAL: For location queries, "in transit" is NOT sufficient - need specific city/facility.

Scoring rubric:
- 1.0: ALL criteria fully satisfied with SPECIFIC, detailed, accurate information
- 0.7-0.9: Most criteria satisfied with specific details, very minor incompleteness
- 0.4-0.6: Some criteria satisfied but lacks specificity or misses key details
- 0.1-0.3: Vague/generic information without specifics ("in transit" without location)
- 0.0: Completely fails to address criteria or provides wrong information

Focus on: Is the response SPECIFIC and COMPLETE? Does it provide EXACT details for every criterion?
Demand precision - vague responses should score 0.3 or lower.
"""
    
    user_context = f"""
<success_criteria>
{success_criteria}
</success_criteria>

<response>
{final_response}
</response>

Evaluate strictly: Does this response FULLY and ACCURATELY meet ALL the success criteria?
Check each criterion individually. Be demanding about completeness.
"""
    
    # Use GPT-4o for better evaluation
    judge_llm = ChatOpenAI(model="gpt-4o", temperature=0)
    grader = judge_llm.with_structured_output(SuccessCriteriaAssessment, method="function_calling")
    
    try:
        assessment = grader.invoke([
            {"role": "system", "content": instructions},
            {"role": "user", "content": user_context}
        ])
        
        return {
            "key": "success_criteria",
            "score": assessment["score"],
            "comment": f"{assessment['reasoning']}"
        }
    except Exception as e:
        return {
            "key": "success_criteria",
            "score": 0.0,
            "comment": f"Evaluation error: {str(e)[:150]}"
        }

print("✓ Defined success_criteria_evaluator")

✓ Defined success_criteria_evaluator


In [12]:
ALL_EVALUATORS = [
    trajectory_match_evaluator,
    success_criteria_evaluator,
    llm_trajectory_evaluator,
    tool_efficiency_evaluator
]
print(f"✓ Loaded {len(ALL_EVALUATORS)} evaluators")
print("   - trajectory_match: Flexible comparison with mode support")
print("   - success_criteria: Boolean scoring for semantic correctness")
print("   - llm_trajectory: Recognizes consolidated tool equivalence")
print("   - tool_efficiency: Measures call count efficiency")

✓ Loaded 4 evaluators
   - trajectory_match: Flexible comparison with mode support
   - success_criteria: Boolean scoring for semantic correctness
   - llm_trajectory: Recognizes consolidated tool equivalence
   - tool_efficiency: Measures call count efficiency


### Agent Wrapper for Structured Output

To enable fair comparisons in our evaluators, we need structured output from the agent:


In [13]:
def run_agent_with_trajectory(agent, query: str) -> dict:
    """
    Runs an agent and returns structured output.
    NO FALLBACKS - fails loudly if expected structure is missing.
    
    Returns:
        {
            "final_response": str,  # The final AI message text
            "trajectory": [         # List of tool calls made
                {"name": str, "args": dict},
                ...
            ]
        }
    """
    # Execute the agent
    result = agent.invoke({"messages": [("user", query)]})
    
    # Extract final AI message
    final_response = ""
    trajectory = []
    
    # NO .get() - fail if messages missing
    messages = result["messages"]
    
    # Extract trajectory from all AI messages with tool calls
    for msg in messages:
        if isinstance(msg, dict) and msg["type"] == "ai":
            # Collect tool calls
            if "tool_calls" in msg and msg["tool_calls"]:
                for tc in msg["tool_calls"]:
                    trajectory.append({
                        "name": tc["name"],  # No .get()
                        "args": tc["args"]   # No .get()
                    })
            # Get final response (last AI message with content)
            if msg["content"]:
                final_response = msg["content"]
        # Handle AIMessage objects
        elif hasattr(msg, 'tool_calls') and msg.tool_calls:
            for tc in msg.tool_calls:
                trajectory.append({
                    "name": tc.name if hasattr(tc, 'name') else tc["name"],
                    "args": tc.args if hasattr(tc, 'args') else tc["args"]
                })
        if hasattr(msg, 'content') and msg.content:
            final_response = msg.content
    
    return {
        "final_response": final_response,
        "trajectory": trajectory
    }

print("✓ Created agent wrapper for structured output (no fallbacks)")


✓ Created agent wrapper for structured output (no fallbacks)


In [14]:
# Extract metrics from local experiment results (no API calls needed)
def get_metrics_from_experiment(experiment) -> Dict[str, float]:
    """
    Extract average metrics from local experiment results.
    NO FALLBACKS - fails loudly if data missing.
    """
    results = list(experiment)
    metrics = {"trajectory_match": [], "llm_trajectory": [], "success_criteria": [], "tool_efficiency": []}
    
    for result in results:
        eval_results = result["evaluation_results"]["results"]  # No .get()
        for eval_result in eval_results:
            key = eval_result.key  # No .get()
            if key in metrics and eval_result.score is not None:
                metrics[key].append(eval_result.score)
    
    # Calculate averages - fail if no scores found
    avg_metrics = {}
    for key, scores in metrics.items():
        if scores:
            avg_metrics[key] = sum(scores) / len(scores)
        else:
            avg_metrics[key] = 0.0
    
    return avg_metrics

print("✓ Defined get_metrics_from_experiment (no fallbacks, updated for success_criteria)")

# Experiments and visualizations will be added below


✓ Defined get_metrics_from_experiment (no fallbacks, updated for success_criteria)


---

## Problem 1: Tool Scaling Degrades Performance

**Hypothesis:** The sheer number of tools is overwhelming the model.

We've seen the production agent struggle. Now let's prove the relationship between tool count and performance.



### Measuring the Problem with LangSmith

Let's evaluate our production agent that has access to 75 tools to see what's actually happening:


In [16]:
# ============================================================================
# EXPERIMENT 1: Production Reality (~75 tools) - Context Confusion
# ============================================================================
print(f"\n🔬 Running Experiment 1: Production agent with {len(all_tools)} tools...")
print("   Expected: LOW performance due to duplicate tools causing confusion")

production_experiment = evaluate(
    lambda inputs: run_agent_with_trajectory(production_agent, inputs["query"]),
    data=dataset_name,
    evaluators=ALL_EVALUATORS,
    experiment_prefix="production-75-tools",
    metadata={"tool_count": len(all_tools), "config": "production-overload"},
)
production_metrics = get_metrics_from_experiment(production_experiment)
print(f"\n✓ Production evaluation complete!")
print(f"   Trajectory Match: {production_metrics['trajectory_match']:.2%}")
print(f"   Success Criteria: {production_metrics['success_criteria']:.2%}")
print(f"   LLM Trajectory: {production_metrics['llm_trajectory']:.2%}")
print(f"   Tool Efficiency: {production_metrics['tool_efficiency']:.2f}")
print("\n   💡 Context confusion: Too many similar tools lead to poor tool selection")


🔬 Running Experiment 1: Production agent with 75 tools...
   Expected: LOW performance due to duplicate tools causing confusion
View the evaluation results for experiment: 'production-75-tools-3f852cf2' at:
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/datasets/46929a17-406b-4431-9eae-cd895dd2265c/compare?selectedSessions=e22306ac-b545-44b9-b372-8bb9dbc60404




0it [00:00, ?it/s]


✓ Production evaluation complete!
   Trajectory Match: 31.25%
   Success Criteria: 56.00%
   LLM Trajectory: 20.00%
   Tool Efficiency: 0.78

   💡 Context confusion: Too many similar tools lead to poor tool selection


### Extract and Visualize Real Metrics from LangSmith


**What LangSmith reveals:**
- Agent calls fraud detection tools on simple order lookups
- Analytics and reporting tools invoked unnecessarily
- More tool calls = longer response times
- Responses include irrelevant information

This is context confusion in action. Let's understand why.


In [21]:
# ============================================================================
# EXPERIMENT 2: Over-Correction (6 specific tools) - Limited Capability
# ============================================================================
print(f"\n🔬 Running Experiment 2: Minimal agent with 6 specific tools...")
print("   Expected: MODERATE performance - efficient but missing capabilities")

minimal_agent = create_agent(
    model=llm,
    tools=shipping_core_tools,
    system_prompt=SHIPPING_SUPPORT_INSTRUCTIONS
)

minimal_experiment = evaluate(
    lambda inputs: run_agent_with_trajectory(minimal_agent, inputs["query"]),
    data=dataset_name,
    evaluators=ALL_EVALUATORS,
    experiment_prefix="minimal-6-tools",
    metadata={"tool_count": 6, "config": "over-correction"},
)

minimal_metrics = get_metrics_from_experiment(minimal_experiment)
print(f"\n✓ Minimal evaluation complete!")
print(f"   Trajectory Match: {minimal_metrics['trajectory_match']:.2%}")
print(f"   Success Criteria: {minimal_metrics['success_criteria']:.2%}")
print(f"   LLM Trajectory: {minimal_metrics['llm_trajectory']:.2%}")
print(f"   Tool Efficiency: {minimal_metrics['tool_efficiency']:.2f}")
print("\n   💡 Overcorrection: Too few tools miss capabilities (returns, carrier incidents, preferences)")



🔬 Running Experiment 2: Minimal agent with 6 specific tools...
   Expected: MODERATE performance - efficient but missing capabilities
View the evaluation results for experiment: 'minimal-6-tools-d567a861' at:
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/datasets/46929a17-406b-4431-9eae-cd895dd2265c/compare?selectedSessions=94cd6269-c8b8-4f37-adc0-88dae7d54c14




0it [00:00, ?it/s]

KeyboardInterrupt: 

---

## Solution 1: Tool Consolidation

**Insight:** Flexible tools with parameters beat many specific tools.

Instead of 75 separate separate tools, create 12 more flexible tools with parameters.


In [22]:
from context_confusion.resources.mock_orders import ORDERS, SHIPMENTS, ORDER_EVENTS
from context_confusion.resources.mock_customers import CUSTOMERS
from context_confusion.resources.mock_carriers import TRACKING_SCANS
from typing import Optional

def get_order_info(
    order_id: str,
    include: List[Literal["status", "tracking", "events", "shipment", "customer"]] = ["status", "shipment"]
) -> dict:
    """
    Retrieve comprehensive order information with flexible parameters.
    
    This consolidated tool replaces multiple specific tools:
    - get_order (basic status)
    - get_shipment (carrier & delivery info)
    - get_order_events (history & tracking events)
    - get_order_summary (combined view)
    - check_order_status (status lookup)
    
    Design advantage: Single flexible call with 'include' parameter replaces 5+ separate tool calls.
    Reduces context confusion by providing one clear choice for order queries.
    
    Args:
        order_id: Order ID to look up  
        include: Information types to fetch in ONE call:
                - "status": Current order state & timestamps
                - "tracking": Tracking number & scan history
                - "events": Full order event log
                - "shipment": Carrier, service level, ETA
                - "customer": Associated customer details
    
    Example:
        get_order_info("12345", include=["status", "tracking", "shipment"])
        Returns complete info in one call instead of 3 separate calls
    """
    oid = order_id.strip().lstrip("#")
    
    if oid not in ORDERS:
        return {"ok": False, "error": f"Order not found: {oid}"}
    
    order = ORDERS[oid]
    result = {"ok": True, "order_id": oid}
    
    if "status" in include:
        result.update({
            "status": order["status"],
            "order_date": order["order_date"],
            "last_update": order["last_update"],
            "total": {"cents": order["total_cents"], "currency": order["currency"]}
        })
    
    if "tracking" in include and order["tracking_number"]:
        tracking_num = order["tracking_number"]
        result["tracking_number"] = tracking_num
        if tracking_num in TRACKING_SCANS:
            result["tracking_scans"] = TRACKING_SCANS[tracking_num]
    
    if "events" in include and oid in ORDER_EVENTS:
        result["events"] = ORDER_EVENTS[oid]
    
    if "shipment" in include and oid in SHIPMENTS:
        shipment = SHIPMENTS[oid]
        result["shipment"] = {
            "carrier": shipment["carrier"],
            "service_level": shipment["service_level"],
            "eta_date": shipment["eta_date"],
            "latest_scan": shipment["latest_scan"],
            "scan_location": shipment["scan_location"],
            "scan_timestamp": shipment["scan_timestamp"],
        }
    
    if "customer" in include:
        customer = CUSTOMERS.get(order["customer_id"], {})
        result["customer"] = {
            "email": customer.get("email"),
            "name": customer.get("name"),
            "tier": customer.get("tier"),
        }
    
    return result

from context_confusion.tools import (
    get_customer_by_email, get_customer, get_customer_preferences, get_billing_info,
    get_tracking_details, get_carrier_info as get_carrier_info_orig, get_carrier_incidents,
    get_shipping_rates, get_carrier_performance, get_return_request, create_return_label,
    cancel_order, hold_order, expedite_order, update_delivery_address, process_refund,
    get_warehouse_info as get_warehouse_info_orig, check_inventory, get_warehouse_incidents,
    create_support_ticket, send_notification, apply_credit, check_fraud_score
)
from context_confusion.resources.mock_customers import CUSTOMERS, CUSTOMER_EMAIL_MAP, CUSTOMER_PREFERENCES, BILLING_INFO
from context_confusion.resources.mock_warehouses import WAREHOUSES, INVENTORY, WAREHOUSE_INCIDENTS
from context_confusion.resources.mock_carriers import CARRIERS, CARRIER_INCIDENTS, RATE_CARDS

# ============================================================================
# SWEET SPOT: 12 Consolidated Tools with Flexible Parameters
# ============================================================================

# Tool 1: get_order_info (already defined above)

# Tool 2: get_customer_info
def get_customer_info(
    identifier: str,
    lookup_by: Literal["email", "customer_id"] = "email",
    include: List[Literal["profile", "preferences", "billing"]] = ["profile"]
) -> dict:
    """
    Retrieve customer information with flexible lookup and detail levels.
    
    This consolidated tool replaces 4 separate customer lookup tools:
    - get_customer (lookup by ID)
    - get_customer_by_email (lookup by email)
    - get_customer_preferences (delivery preferences)
    - get_billing_info (payment & billing details)
    
    Design advantage: Single tool handles both lookup methods AND multiple info types.
    Agent doesn't need to choose between "get_customer" vs "get_customer_by_email" - 
    just specify lookup_by parameter. Reduces decision fatigue and tool confusion.
    
    Args:
        identifier: Customer email or ID
        lookup_by: How to find customer - "email" or "customer_id"
        include: Information types to retrieve:
                - "profile": Name, email, tier, phone, location
                - "preferences": Delivery instructions, notifications
                - "billing": Payment methods, billing address
    
    Example:
        get_customer_info("user@example.com", lookup_by="email", include=["profile", "preferences"])
        Replaces: get_customer_by_email() + get_customer_preferences() (2 calls → 1)
    """
    if lookup_by == "email":
        email = identifier.strip().lower()
        if email not in CUSTOMER_EMAIL_MAP:
            return {"ok": False, "error": "Customer not found"}
        cid = CUSTOMER_EMAIL_MAP[email]
    else:
        cid = identifier.strip()
        if cid not in CUSTOMERS:
            return {"ok": False, "error": "Customer not found"}
    
    customer = CUSTOMERS[cid]
    result = {"ok": True, "customer_id": cid}
    
    if "profile" in include:
        result["profile"] = {
            "email": customer["email"],
            "name": customer["name"],
            "tier": customer["tier"],
            "phone": customer["phone"],
            "location": customer["location"]
        }
    
    if "preferences" in include and cid in CUSTOMER_PREFERENCES:
        result["preferences"] = CUSTOMER_PREFERENCES[cid]
    
    if "billing" in include and cid in BILLING_INFO:
        result["billing"] = BILLING_INFO[cid]
    
    return result

# Tool 3: get_tracking_info
def get_tracking_info(
    tracking_number: str,
    include: List[Literal["scans", "events", "eta"]] = ["scans"]
) -> dict:
    """Retrieve detailed tracking information. Replaces: get_tracking_details (keeps but enhances)"""
    return get_tracking_details(tracking_number)

# Tool 4: get_carrier_info
def get_carrier_info(
    carrier_id: Optional[str] = None,
    include: List[Literal["details", "incidents", "rates", "performance"]] = ["details"],
    date: Optional[str] = None
) -> dict:
    """
    Retrieve carrier information including incidents, rates, and performance.
    
    This consolidated tool replaces 4 carrier-related tools:
    - get_carrier_info (basic carrier details)
    - get_carrier_incidents (service disruptions)
    - get_shipping_rates (cost estimation)
    - get_carrier_performance (metrics & SLA)
    
    Design advantage: All carrier queries go through ONE tool with flexible parameters.
    Critical for diagnosing delivery issues where you need both carrier details AND incidents.
    
    Args:
        carrier_id: Carrier to look up (optional for incident checks)
        include: Information types:
                - "details": Carrier name, services, contact
                - "incidents": Service disruptions & delays
                - "rates": Shipping cost information
                - "performance": On-time delivery metrics
        date: Required for incident checks (YYYY-MM-DD format)
    
    Example:
        get_carrier_info("ups", include=["details", "incidents"], date="2024-12-20")
        One call replaces: get_carrier_info("ups") + get_carrier_incidents("2024-12-20")
    """
    result = {"ok": True}
    
    if carrier_id and "details" in include:
        carrier_result = get_carrier_info_orig(carrier_id)
        if carrier_result["ok"]:
            result["carrier"] = carrier_result["data"]
    
    if "incidents" in include and date:
        incidents_result = get_carrier_incidents(date)
        if incidents_result["ok"]:
            result["incidents"] = incidents_result["data"]
    
    if "rates" in include:
        result["rate_info"] = "Use get_shipping_rates with origin/destination"
    
    return result

# Tool 5: get_return_info
def get_return_info(
    order_id: str,
    include: List[Literal["request", "label", "status"]] = ["request"]
) -> dict:
    """
    Retrieve or manage return information.
    
    This consolidated tool replaces 2 return-related tools:
    - get_return_request (check return status)
    - create_return_label (generate label)
    
    Design advantage: All return queries through one tool. Agent doesn't need to decide
    between multiple return tools - just specify what info is needed via 'include'.
    
    Args:
        order_id: Order to check returns for
        include: Information types:
                - "request": Return request status
                - "label": Return shipping label
                - "status": Current return state
    """
    return get_return_request(order_id)

# Tool 6: manage_order
def manage_order(
    order_id: str,
    action: Literal["cancel", "hold", "expedite", "update_address"],
    **params
) -> dict:
    """
    Perform order management actions.
    
    This consolidated tool replaces 4 order modification tools:
    - cancel_order (cancel processing)
    - hold_order (pause fulfillment)
    - expedite_order (upgrade shipping speed)
    - update_delivery_address (change destination)
    
    Design advantage: Single "manage_order" tool with 'action' parameter replaces 4 tools.
    Agent chooses the action, not the tool - clearer intent, less confusion.
    Extensible design allows adding new actions without new tools.
    
    Args:
        order_id: Order to modify
        action: Management action to perform
        **params: Action-specific parameters (reason, new_address, etc)
    
    Example:
        manage_order("12345", action="expedite", new_shipping_method="express")
        Replaces: expedite_order("12345", "express")
    """
    if action == "cancel":
        return cancel_order(order_id, params.get("reason", "Customer request"))
    elif action == "hold":
        return hold_order(order_id, params.get("reason", "Hold requested"))
    elif action == "expedite":
        return expedite_order(order_id, params.get("new_shipping_method", "express"))
    elif action == "update_address":
        return update_delivery_address(order_id, params.get("new_address", {}))
    return {"ok": False, "error": "Invalid action"}

# Tool 7-12: Keep specific action tools as-is
# process_refund, get_warehouse_info_consolidated, create_support_ticket, 
# send_notification, apply_credit, check_fraud_score

def get_warehouse_info_consolidated(
    include: List[Literal["locations", "inventory", "incidents"]] = ["locations"],
    sku: Optional[str] = None
) -> dict:
    """
    Retrieve warehouse and inventory information.
    
    This consolidated tool replaces 3 warehouse tools:
    - get_warehouse_info (facility locations)
    - check_inventory (stock levels)
    - get_warehouse_incidents (facility issues)
    
    Design advantage: All warehouse queries through one flexible tool.
    Particularly useful for stock availability checks where you might need
    both inventory levels AND warehouse incidents affecting fulfillment.
    
    Args:
        include: Information types:
                - "locations": Warehouse facilities
                - "inventory": Stock levels (all or by SKU)
                - "incidents": Facility disruptions
        sku: Optional SKU to filter inventory check
    
    Example:
        get_warehouse_info_consolidated(include=["inventory", "incidents"], sku="WIDGET-123")
        Replaces: check_inventory("WIDGET-123") + get_warehouse_incidents()
    """
    result = {"ok": True}
    
    if "locations" in include:
        result["warehouses"] = WAREHOUSES
    
    if "inventory" in include:
        if sku:
            result["inventory"] = {k: v for k, v in INVENTORY.items() if k == sku}
        else:
            result["inventory"] = INVENTORY
    
    if "incidents" in include:
        result["incidents"] = WAREHOUSE_INCIDENTS
    
    return result

# Final consolidated tools list (12 tools)
consolidated_tools = [
    get_order_info,
    get_customer_info,
    get_tracking_info,
    get_carrier_info,
    get_return_info,
    manage_order,
    process_refund,
    get_warehouse_info_consolidated,
    create_support_ticket,
    send_notification,
    apply_credit,
    check_fraud_score,
]

print(f"✓ Created {len(consolidated_tools)} consolidated tools")
print("   These 12 tools replace ~20 commonly used specific tools")


✓ Created 12 consolidated tools
   These 12 tools replace ~20 commonly used specific tools


In [23]:
# ============================================================================
# EXPERIMENT 3: 12 consolidated tools - Balance of tool number
# ============================================================================
from context_confusion.instructions import CONSOLIDATED_INSTRUCTIONS

print(f"\n🔬 Running Experiment 3: Optimal agent with 12 consolidated tools...")
print("   Expected: HIGH performance - clarity + capability")

optimal_agent = create_agent(
    model=llm,
    tools=consolidated_tools,
    system_prompt=CONSOLIDATED_INSTRUCTIONS
)

optimal_experiment = evaluate(
    lambda inputs: run_agent_with_trajectory(optimal_agent, inputs["query"]),
    data=dataset_name,
    evaluators=ALL_EVALUATORS,
    experiment_prefix="optimal-12-consolidated",
    metadata={"tool_count": 12, "config": "sweet-spot"},
)

optimal_metrics = get_metrics_from_experiment(optimal_experiment)
print(f"\n✓ Optimal evaluation complete!")
print(f"   Trajectory Match: {optimal_metrics['trajectory_match']:.2%}")
print(f"   Success Criteria: {optimal_metrics['success_criteria']:.2%}")
print(f"   LLM Trajectory: {optimal_metrics['llm_trajectory']:.2%}")
print(f"   Tool Efficiency: {optimal_metrics['tool_efficiency']:.2f}")
print("\n   ✅ Sweet spot: Consolidated tools with parameters provide clarity AND capability!")


🔬 Running Experiment 3: Optimal agent with 12 consolidated tools...
   Expected: HIGH performance - clarity + capability
View the evaluation results for experiment: 'optimal-12-consolidated-f1af69ee' at:
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/datasets/46929a17-406b-4431-9eae-cd895dd2265c/compare?selectedSessions=c64b81e3-e29e-444e-8108-fa8be551309b




0it [00:00, ?it/s]


✓ Optimal evaluation complete!
   Trajectory Match: 0.00%
   Success Criteria: 83.00%
   LLM Trajectory: 70.00%
   Tool Efficiency: 0.90

   ✅ Sweet spot: Consolidated tools with parameters provide clarity AND capability!


## Tool Count Comparison

The results reveal a clear U-curve - performance suffers at both extremes:


In [ ]:
# Note: Optimal experiment (Experiment 3 above) already tests consolidated tools
# No separate test needed here


In [ ]:
# Metrics already extracted in Experiment 3 above

In [ ]:
# Comparison visualization showing clear accuracy wins
configs = ["75 Tools\n(Overload)", "6 Tools\n(Limited)", "12 Tools\n(Consolidated)"]
trajectory_scores = [
    production_metrics["trajectory_match"],
    minimal_metrics["trajectory_match"],
    optimal_metrics["trajectory_match"]
]
success_scores = [
    production_metrics["success_criteria"],
    minimal_metrics["success_criteria"],
    optimal_metrics["success_criteria"]
]
efficiency_scores = [
    production_metrics["tool_efficiency"],
    minimal_metrics["tool_efficiency"],
    optimal_metrics["tool_efficiency"]
]

# Create grouped bar chart
fig = go.Figure()
fig.add_trace(go.Bar(
    name='Trajectory Match',
    x=configs,
    y=trajectory_scores,
    marker_color='#1f77b4'
))
fig.add_trace(go.Bar(
    name='Success Criteria',
    x=configs,
    y=success_scores,
    marker_color='#2ca02c'
))

fig.update_layout(
    title="Finding the Sweet Spot: Consolidated Tools Win on Accuracy",
    yaxis_title="Score",
    barmode='group',
    height=450,
    showlegend=True,
    yaxis=dict(range=[0, 1.0])
)

fig.show()

# Create summary table
comparison_df = pd.DataFrame({
    "Configuration": configs,
    "Tool Count": [len(all_tools), 6, 12],
    "Trajectory Match": [f"{s:.1%}" for s in trajectory_scores],
    "Success Criteria": [f"{s:.1%}" for s in success_scores],
    "Tool Efficiency": [f"{s:.2f}" for s in efficiency_scores],
})

display(comparison_df)

print("\n📊 Key Findings:")
print(f"   75 tools: {trajectory_scores[0]:.1%} trajectory match, {success_scores[0]:.1%} success criteria")
print(f"   6 tools:  {trajectory_scores[1]:.1%} trajectory match, {success_scores[1]:.1%} success criteria")
print(f"   12 tools: {trajectory_scores[2]:.1%} trajectory match, {success_scores[2]:.1%} success criteria")
print("\n   ✅ Sweet Spot Achieved:")
print("   Consolidated tools outperform both extremes by reducing context confusion")
print("   Flexible parameters provide clarity (better tool selection) AND capability (full coverage)")


**LangSmith shows us:** Fewer, focused tools perform better than many scattered tools.


In [ ]:
# Consolidated tool dataset - same queries, slightly different expected trajectory
consolidated_dataset_name = "shipping-support-queries-consolidated"

test_cases_consolidated = [
    {
        "query": "I'm user@example.com, what's the status of order #84721?",
        "final_response": "Order #84721 is currently in transit.",
        "trajectory": [
            {"tool": "get_customer_by_email", "args": {"email": "user@example.com"}},
            {"tool": "get_order_info", "args": {"order_id": "84721", "include": ["status"]}}
        ],
        "success_criteria": "Should return order status: IN_TRANSIT"
    },
    {
        "query": "Track order 99002",
        "final_response": "Order #99002 has been delivered successfully.",
        "trajectory": [
            {"tool": "get_order_info", "args": {"order_id": "99002", "include": ["status", "shipment"]}}
        ],
        "success_criteria": "Should show DELIVERED status"
    },
    {
        "query": "I'm buyer@uk-shop.co.uk, why is order #23456 delayed?",
        "final_response": "Order #23456 is experiencing a customs delay.",
        "trajectory": [
            {"tool": "get_customer_by_email", "args": {"email": "buyer@uk-shop.co.uk"}},
            {"tool": "get_order_info", "args": {"order_id": "23456", "include": ["status", "shipment"]}}
        ],
        "success_criteria": "Should mention customs delay"
    },
    {
        "query": "I'm ops@widget.io and I need urgent help with order #10015",
        "final_response": "As a platinum tier customer, I'll prioritize your request for order #10015.",
        "trajectory": [
            {"tool": "get_customer_by_email", "args": {"email": "ops@widget.io"}},
            {"tool": "get_order_info", "args": {"order_id": "10015", "include": ["status", "customer"]}}
        ],
        "success_criteria": "Should recognize platinum tier customer"
    },
    {
        "query": "What's the tracking number for order 84721?",
        "final_response": "The tracking number for order #84721 is TRK84721ABC.",
        "trajectory": [
            {"tool": "get_order_info", "args": {"order_id": "84721", "include": ["tracking"]}}
        ],
        "success_criteria": "Should return tracking number"
    },
    {
        "query": "When will order #98765 arrive?",
        "final_response": "Order #98765 is estimated to arrive on March 20, 2024.",
        "trajectory": [
            {"tool": "get_order_info", "args": {"order_id": "98765", "include": ["shipment"]}}
        ],
        "success_criteria": "Should provide ETA date"
    },
    {
        "query": "I'm manager@euro-corp.eu, can you tell me about order #45678?",
        "final_response": "Order #45678 status available. Please note EU privacy requirements.",
        "trajectory": [
            {"tool": "get_customer_by_email", "args": {"email": "manager@euro-corp.eu"}},
            {"tool": "get_order_info", "args": {"order_id": "45678", "include": ["status", "customer"]}}
        ],
        "success_criteria": "Should note EU customer privacy"
    },
    {
        "query": "Show me the delivery history for order 84721",
        "final_response": "Order #84721 delivery history shows multiple scan events.",
        "trajectory": [
            {"tool": "get_order_info", "args": {"order_id": "84721", "include": ["tracking", "events"]}}
        ],
        "success_criteria": "Should show scan history"
    },
    {
        "query": "What carrier is handling order #23456?",
        "final_response": "Order #23456 is being shipped by Royal Mail.",
        "trajectory": [
            {"tool": "get_order_info", "args": {"order_id": "23456", "include": ["shipment"]}}
        ],
        "success_criteria": "Should return carrier: Royal Mail"
    },
    {
        "query": "Has order #11111 been delivered?",
        "final_response": "Order #11111 has been returned.",
        "trajectory": [
            {"tool": "get_order_info", "args": {"order_id": "11111", "include": ["status"]}}
        ],
        "success_criteria": "Should return status: RETURNED"
    },
]

# Create consolidated dataset - force recreate to ensure correct structure
try:
    existing = client.read_dataset(dataset_name=consolidated_dataset_name)
    client.delete_dataset(dataset_id=existing.id)
    print(f"✓ Deleted old consolidated dataset")
except:
    pass

consolidated_dataset = client.create_dataset(
    dataset_name=consolidated_dataset_name,
    description="Test queries with consolidated tool references"
)
for case in test_cases_consolidated:
    client.create_example(
        inputs={"query": case["query"]},
        outputs={
            "final_response": case["final_response"],
            "trajectory": case["trajectory"],
            "success_criteria": case["success_criteria"]
        },
        dataset_id=consolidated_dataset.id
    )
print(f"✓ Created consolidated dataset with {len(test_cases_consolidated)} examples")
print(f"   View dataset: https://smith.langchain.com/o/{client._get_tenant_id()}/datasets/{consolidated_dataset.id}")



---

## Problem 2: Irrelevant Tools Cause Distraction

**Hypothesis:** It's not just quantity - irrelevant tools are the real problem.

Test different ratios of relevant:irrelevant tools (total ~20).


In [ ]:
# Create noise ratio configurations
noise_configs = [
    {"name": "100-pct-relevant", "tools": shipping_core_tools + carrier_tools + returns_tools + warehouse_tools[:2]},
    {"name": "75-pct-relevant", "tools": shipping_core_tools + carrier_tools + returns_tools + fraud_tools[:3]},
    {"name": "50-pct-relevant", "tools": shipping_core_tools + carrier_tools + analytics_tools + marketing_tools[:4]},
    {"name": "25-pct-relevant", "tools": shipping_core_tools + fraud_tools + analytics_tools + marketing_tools + vendor_tools[:2]},
]

print("Noise ratio configurations:")
for config in noise_configs:
    print(f"  {config['name']}: {len(config['tools'])} total tools")

# Run experiments
for config in noise_configs:
    print(f"\nTesting {config['name']}...")
    
    agent = create_agent(
        model=llm,
        tools=config["tools"],
        system_prompt=SHIPPING_SUPPORT_INSTRUCTIONS
    )
    
    evaluate(
        lambda inputs: run_agent_with_trajectory(agent, inputs["query"]),
        data=dataset_name,
        evaluators=ALL_EVALUATORS,
        experiment_prefix=f"noise-{config['name']}",
        metadata={"config": config["name"]},
    )
    
    print(f"  ✓ Complete")

print("\n✓ All noise ratio experiments complete!")


### Analyzing the Impact of Irrelevant Tools

Let's extract and visualize the results to see how noise affects performance:


In [ ]:
# Extract metrics from noise ratio experiments
noise_percentages = [100, 75, 50, 25]
noise_configs_names = ["100-pct-relevant", "75-pct-relevant", "50-pct-relevant", "25-pct-relevant"]

relevance_by_noise = []
unnecessary_by_noise = []

for config_name in noise_configs_names:
    exp_name = f"noise-{config_name}"
    metrics = get_metrics_from_experiment(exp_name)
    relevance_by_noise.append(metrics.get("trajectory_match", 0))
    unnecessary_by_noise.append(metrics.get("correctness", 0))

# Visualize the impact
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=noise_percentages,
    y=relevance_by_noise,
    mode='lines+markers',
    name='Trajectory Match',
    line=dict(color='#1f77b4', width=3),
    marker=dict(size=10)
))
fig.add_trace(go.Scatter(
    x=noise_percentages,
    y=unnecessary_by_noise,
    mode='lines+markers',
    name='Correctness',
    line=dict(color='#2ca02c', width=3),
    marker=dict(size=10),
    yaxis='y2'
))

fig.update_layout(
    title="Impact of Irrelevant Tools on Agent Performance",
    xaxis_title="% Relevant Tools",
    yaxis_title="Trajectory Match Score",
    yaxis2=dict(
        title="Correctness Score",
        overlaying='y',
        side='right'
    ),
    hovermode='x unified',
    height=400
)

fig.show()

# Create comparison table
noise_comparison = {
    "% Relevant": noise_percentages,
    "Trajectory Match": relevance_by_noise if relevance_by_noise[0] > 0 else ["Run eval"] * 4,
    "Correctness": unnecessary_by_noise if unnecessary_by_noise[0] > 0 else ["Run eval"] * 4,
}

df_noise = pd.DataFrame(noise_comparison)
display(df_noise)

print("\n📊 Key Findings:")
if relevance_by_noise[0] > 0:
    print(f"   100% relevant tools: {relevance_by_noise[0]:.2f} trajectory match, {unnecessary_by_noise[0]:.2f} correctness")
    print(f"   25% relevant tools: {relevance_by_noise[3]:.2f} trajectory match, {unnecessary_by_noise[3]:.2f} correctness")
    degradation = ((relevance_by_noise[0] - relevance_by_noise[3]) / relevance_by_noise[0] * 100) if relevance_by_noise[0] > 0 else 0
    print(f"   Degradation: {degradation:.0f}% drop in trajectory match")
else:
    print("   Run evaluations above to see actual metrics")
print("   - Even with constant tool count (~20), noise degrades performance")
print("   - Irrelevant tools lead to incorrect tool calls")
print("   - Quality of tools matters as much as quantity")


---

## Solution 2: Smart Tool Filtering with Routing

**Insight:** Classify query intent, then provide only relevant tools.


In [ ]:
# Simple query router
def classify_query_intent(query: str) -> Literal["shipping", "returns", "billing", "warehouse"]:
    """Classify intent - in production use an LLM."""
    query_lower = query.lower()
    
    if any(word in query_lower for word in ["return", "refund", "send back"]):
        return "returns"
    elif any(word in query_lower for word in ["bill", "charge", "payment"]):
        return "billing"
    elif any(word in query_lower for word in ["stock", "inventory", "warehouse"]):
        return "warehouse"
    else:
        return "shipping"

def get_tools_for_intent(intent: str) -> list:
    """Return tools for intent."""
    if intent == "shipping":
        return shipping_core_tools + carrier_tools
    elif intent == "returns":
        return shipping_core_tools + returns_tools
    elif intent == "billing":
        return shipping_core_tools + billing_tools
    elif intent == "warehouse":
        return shipping_core_tools + warehouse_tools
    return shipping_core_tools

print("✓ Created intent-based router")

# Test routed agent
def run_routed_agent_with_trajectory(inputs: dict) -> dict:
    """Run routed agent with intent-based tool selection and return structured output."""
    query = inputs["query"]
    intent = classify_query_intent(query)
    tools = get_tools_for_intent(intent)
    
    agent = create_agent(
        model=llm,
        tools=tools,
        system_prompt=SHIPPING_SUPPORT_INSTRUCTIONS
    )
    
    return run_agent_with_trajectory(agent, query)

routed_results = evaluate(
    run_routed_agent_with_trajectory,
    data=dataset_name,
    evaluators=ALL_EVALUATORS,
    experiment_prefix="solution-routed",
    metadata={"config": "routed"},
)

print(f"\n✓ Routed agent evaluation complete!")


### Comparing Routing vs Noise

How does smart routing compare to the worst-case noise scenario?


In [ ]:
# Compare routing solution vs worst noise configuration
try:
    worst_noise_metrics = get_metrics_from_experiment("noise-25-pct-relevant")
    routed_metrics = get_metrics_from_experiment("solution-routed")
    
    routing_comparison = {
        "Configuration": ["25% Relevant Tools (No Routing)", "Smart Routing"],
        "Trajectory Match": [worst_noise_metrics.get("trajectory_match", 0), routed_metrics.get("trajectory_match", 0)],
        "Correctness": [worst_noise_metrics.get("correctness", 0), routed_metrics.get("correctness", 0)],
        "Tool Efficiency": [worst_noise_metrics.get("tool_efficiency", 0), routed_metrics.get("tool_efficiency", 0)],
    }
    
    # Create bar chart comparison
    df_routing = pd.DataFrame(routing_comparison)
    
    fig = go.Figure()
    fig.add_trace(go.Bar(
        name='25% Relevant (No Routing)',
        x=['Trajectory Match', 'Correctness'],
        y=[worst_noise_metrics.get("trajectory_match", 0), worst_noise_metrics.get("correctness", 0)],
        marker_color='#d62728'
    ))
    fig.add_trace(go.Bar(
        name='Smart Routing',
        x=['Trajectory Match', 'Correctness'],
        y=[routed_metrics.get("trajectory_match", 0), routed_metrics.get("correctness", 0)],
        marker_color='#2ca02c'
    ))
    
    fig.update_layout(
        title="Routing Solution vs Worst Noise Configuration",
        yaxis_title="Score",
        barmode='group',
        height=400
    )
    
    fig.show()
    
    display(df_routing)
    
    print("\n✅ Smart routing eliminates noise:")
    if routed_metrics.get("trajectory_match", 0) > 0:
        improvement = ((routed_metrics.get("trajectory_match", 0) - worst_noise_metrics.get("trajectory_match", 0)) / worst_noise_metrics.get("trajectory_match", 1) * 100) if worst_noise_metrics.get("trajectory_match", 0) > 0 else 0
        print(f"   Trajectory match improvement: {improvement:.0f}%")
        print(f"   Correctness improved from {worst_noise_metrics.get('correctness', 0):.2f} to {routed_metrics.get('correctness', 0):.2f}")
    else:
        print("   Run evaluations above to see actual metrics")
    print("   - Intent classification filters out irrelevant tool domains")
    print("   - Agent sees only tools needed for the query type")
    print("   - Maintains performance by keeping context focused")
    
except Exception as e:
    print(f"Run the evaluations above first to see comparison. Error: {e}")
    routing_comparison = {
        "Configuration": ["25% Relevant Tools (No Routing)", "Smart Routing"],
        "Trajectory Match": ["Run eval", "Run eval"],
        "Correctness": ["Run eval", "Run eval"],
        "Tool Efficiency": ["Run eval", "Run eval"],
    }
    df_routing = pd.DataFrame(routing_comparison)
    display(df_routing)


---

## Problem 3: Instruction Bloat Hurts Too

**Hypothesis:** Verbose instructions degrade performance.

Add evaluator to check instruction following:


In [ ]:
# Expected tools checker is now replaced by trajectory_match_evaluator


In [ ]:
# Test with bloated instructions
clean_instructions = f"{SHIPPING_SUPPORT_INSTRUCTIONS}\n\n{CARRIER_MANAGEMENT_INSTRUCTIONS}"
noisy_instructions = f"{clean_instructions}\n\n{IRRELEVANT_INSTRUCTIONS}"

print(f"Clean: {len(clean_instructions):,} chars")
print(f"Noisy: {len(noisy_instructions):,} chars")
print(f"Bloat: {len(noisy_instructions) / len(clean_instructions):.1f}x")

noisy_agent = create_agent(
    model=llm,
    tools=shipping_core_tools,
    system_prompt=noisy_instructions
)

evaluate(
    lambda inputs: run_agent_with_trajectory(noisy_agent, inputs["query"]),
    data=dataset_name,
    evaluators=ALL_EVALUATORS,
    experiment_prefix="problem-noisy-instructions",
    metadata={"tool_count": 6, "instruction_length": len(noisy_instructions)},
)

print("✓ Evaluation complete!")


### Analyzing Instruction Bloat Impact

Let's see how extra instructions affected performance:


In [ ]:
# Extract and compare instruction bloat impact
try:
    clean_baseline_metrics = get_metrics_from_experiment("scaling-6-tools")
    bloated_metrics = get_metrics_from_experiment("problem-noisy-instructions")
    
    # Create comparison visualization
    metrics_to_compare = ['Trajectory Match', 'Correctness', 'Tool Efficiency']
    clean_scores = [
        clean_baseline_metrics.get("trajectory_match", 0),
        clean_baseline_metrics.get("correctness", 0),
        clean_baseline_metrics.get("tool_efficiency", 1.0)  # Baseline expected to be 1.0
    ]
    bloated_scores = [
        bloated_metrics.get("trajectory_match", 0),
        bloated_metrics.get("correctness", 0),
        bloated_metrics.get("tool_efficiency", 0)
    ]
    
    fig = go.Figure()
    fig.add_trace(go.Bar(
        name='Clean Instructions',
        x=metrics_to_compare,
        y=clean_scores,
        marker_color='#2ca02c'
    ))
    fig.add_trace(go.Bar(
        name='Bloated Instructions',
        x=metrics_to_compare,
        y=bloated_scores,
        marker_color='#d62728'
    ))
    
    fig.update_layout(
        title="Impact of Instruction Bloat on Agent Performance",
        yaxis_title="Score",
        barmode='group',
        height=400
    )
    
    fig.show()
    
    # Create comparison table
    instruction_impact = {
        "Metric": ["Trajectory Match", "Correctness", "Tool Efficiency"],
        "Clean Instructions": [
            f"{clean_baseline_metrics.get('trajectory_match', 0):.2f}",
            f"{clean_baseline_metrics.get('correctness', 0):.2f}",
            f"{clean_baseline_metrics.get('tool_efficiency', 1.0):.2f}"
        ],
        "Bloated Instructions": [
            f"{bloated_metrics.get('trajectory_match', 0):.2f}",
            f"{bloated_metrics.get('correctness', 0):.2f}",
            f"{bloated_metrics.get('tool_efficiency', 0):.2f}"
        ]
    }
    
    df_instructions = pd.DataFrame(instruction_impact)
    display(df_instructions)
    
    print("\n📉 Key Findings on Instruction Bloat:")
    if bloated_metrics.get("trajectory_match", 0) > 0:
        traj_drop = ((clean_baseline_metrics.get("trajectory_match", 0) - bloated_metrics.get("trajectory_match", 0)) / clean_baseline_metrics.get("trajectory_match", 1) * 100) if clean_baseline_metrics.get("trajectory_match", 0) > 0 else 0
        corr_drop = ((clean_baseline_metrics.get("correctness", 0) - bloated_metrics.get("correctness", 0)) / clean_baseline_metrics.get("correctness", 1) * 100) if clean_baseline_metrics.get("correctness", 0) > 0 else 0
        print(f"   Trajectory match degraded by {traj_drop:.0f}%")
        print(f"   Correctness degraded by {corr_drop:.0f}%")
        print(f"   Tool efficiency: {bloated_metrics.get('tool_efficiency', 0):.2f}")
    else:
        print("   Run evaluations above to see actual metrics")
    print("   - Extra instructions from irrelevant domains confuse the agent")
    print("   - Agent may miss obvious tool choices")
    print("   - Focused instructions lead to better tool selection")
    
except Exception as e:
    print(f"Run the evaluations above first to see comparison. Error: {e}")
    instruction_impact = {
        "Metric": ["Trajectory Match", "Correctness", "Tool Efficiency"],
        "Clean Instructions": ["Run eval", "Run eval", "Run eval"],
        "Bloated Instructions": ["Run eval", "Run eval", "Run eval"]
    }
    df_instructions = pd.DataFrame(instruction_impact)
    display(df_instructions)


---

## Solution 3: Instruction Pruning

**Insight:** Keep only essential instructions. LangSmith A/B testing validates.


In [ ]:
# Extract and compare
try:
    clean_metrics = get_metrics_from_experiment("scaling-6-tools")
    noisy_metrics = get_metrics_from_experiment("problem-noisy-instructions")
    
    instruction_comparison = {
        "Instructions": ["Clean (focused)", "Bloated (all domains)"],
        "Tool Count": [6, 6],
        "Trajectory Match": [clean_metrics.get("trajectory_match", 0), noisy_metrics.get("trajectory_match", 0)],
        "Correctness": [clean_metrics.get("correctness", 0), noisy_metrics.get("correctness", 0)],
    }
except:
    instruction_comparison = {
        "Instructions": ["Clean (focused)", "Bloated (all domains)"],
        "Tool Count": [6, 6],
        "Trajectory Match": ["Run eval", "Run eval"],
        "Correctness": ["Run eval", "Run eval"],
    }

df_instructions = pd.DataFrame(instruction_comparison)
display(df_instructions)

print("\n✅ Minimal instructions maintain or improve performance")
print("   - LangSmith A/B testing validates focused instructions")


---

## Key Takeaways

### The Sweet Spot Is Real

Our experiments demonstrate a clear U-curve:

- **75 tools (overload):** Context confusion from overlapping choices leads to poor performance
- **6 tools (minimal):** Over-correction leaves gaps - can't handle all query types
- **12 tools (consolidated):** Sweet spot with flexible parameters delivers full capability AND clarity

**The winning approach:** Intelligent consolidation at the right level of abstraction.

### What Makes Consolidated Tools Work

1. **Flexible parameters over many specific tools** - One `get_order_info` with `include` parameter beats five separate getter functions

2. **Reduces cognitive load** - 12 well-designed tools are easier to navigate than 75 overlapping options

3. **Maintains full capability** - Parameter-based design covers all use cases without sacrificing functionality

4. **Measurable with LangSmith** - Custom evaluators (trajectory match, correctness, efficiency) quantify the improvement

### Professional Implementation Strategy

When facing context confusion in production:

1. **Start with measurement** - Use LangSmith to quantify the problem (don't guess)
2. **Identify tool clusters** - Group tools that serve similar purposes
3. **Design consolidated versions** - Add parameters for flexibility instead of creating new tools
4. **Validate with evals** - Ensure consolidated tools match or exceed original performance
5. **Iterate to sweet spot** - Find the balance between too many and too few

### Real-World Impact

By finding the sweet spot:
- **Better performance** - Higher accuracy, fewer mistakes from tool confusion
- **Lower costs** - Fewer tokens in context, faster inference
- **Easier maintenance** - Consolidated, well-organized toolsets
- **Better UX** - Faster, more accurate responses for users
- **Scalable** - Room to add capabilities without degrading performance

---

**Context confusion isn't solved by minimalism - it's solved by intelligent design. Use LangSmith to measure, then optimize to your sweet spot.**
